# Limpeza e Preparação de Dados

Embora as pessoas frequentemente associem a limpeza de dados a bibliotecas de dados Python como o Pandas, o SQL pode, na verdade, fazer bastante e superar o desafio. Abordaremos algumas funções essenciais para limpeza de dados, incluindo o tratamento de valores nulos, lógica condicional e operações com strings.

Nesta seção, vamos praticar com o [conjunto de dados de colisão de pássaros da FAA](https://wildlife.faa.gov/home), que é um conjunto de dados bastante complexo com algumas oportunidades de limpeza. Se você quiser aprender mais sobre esse conjunto de dados, veja o [curso completo de análise exploratória de dados no Anaconda](https://learning.anaconda.cloud/exploratory-data-analysis-eda-with-python) usando o Pandas.

Vamos importar o banco de dados que preparei e converti para SQLite, apenas para registros desde 2015.

In [ ]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('bird_strike.db')

pd.read_sql("SELECT * FROM BIRD_STRIKE_FAA", conn)


Vamos falar primeiro sobre expressões de caso e, em seguida, sobre valores nulos.

## Expressões de Caso

Digamos que eu selecione estes campos da tabela `BIRD_STRIKE_FAA`. Quero sinalizar quaisquer registros que envolvam um Boeing 737, que possui diversas variantes. Vamos usar um operador curinga `LIKE` para fazer isso.

In [ ]:
sql = """ SELECT 
AIRPORT_ID,
AIRPORT,
AIRCRAFT

FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT LIKE '%737%'
"""

pd.read_sql(sql, conn) 

Se quisermos verificar quais valores estão sendo capturados com `%737%`, podemos usar um operador `DISTINCT`.

In [ ]:
sql = """ 
SELECT DISTINCT AIRCRAFT
FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT LIKE '%737%'
"""

pd.read_sql(sql, conn) 

Agora, quero comparar os incidentes do Boeing 737 com os incidentes do Airbus A320. Podemos categorizá-los dessa forma usando uma expressão `CASE` e usar a ramificação `ELSE` para sinalizar todo o resto como `OTHER`.

In [ ]:
sql = """ 
SELECT INDEX_NR, 
AIRCRAFT, 
INCIDENT_DATE,
STATE, 
AIRPORT_ID, 

CASE 
    WHEN AIRCRAFT LIKE '%737%' THEN 'Boeing 737'
    WHEN AIRCRAFT LIKE 'A-320%' THEN 'Airbus A-320'
    ELSE 'Other'
END AIRCRAFT_OF_INTEREST_FLAG 

FROM BIRD_STRIKE_FAA

"""

pd.read_sql(sql, conn) 

Para ter uma ideia de quantos registros receberam cada uma dessas sinalizações, vamos usar `COUNT(*)` pareado com `GROUP BY`.

In [ ]:
sql = """ 
SELECT 

CASE 
    WHEN AIRCRAFT LIKE '%737%' THEN 'Boeing 737-related'
    WHEN AIRCRAFT LIKE 'A-320%' THEN 'Airbus A-320-related'
    ELSE 'Other'
END AIRCRAFT_OF_INTEREST_FLAG, 

COUNT(*) AS RECORD_COUNT

FROM BIRD_STRIKE_FAA

GROUP BY AIRCRAFT_OF_INTEREST_FLAG

"""

pd.read_sql(sql, conn) 

A expressão `CASE` é útil para todos os tipos de transformações e limpeza de dados. Vamos falar sobre valores nulos a seguir.

## Identificando Valores Nulos

Vamos analisar alguns campos da tabela. Observe como os campos `RUNWAY` e `PHASE_OF_FLIGHT` têm alguns valores `None` e as colunas `HEIGHT`, `SPEED` e `DISTANCE` têm `NaN`.

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

O pandas está, na verdade, transformando valores `NULL` do banco de dados SQLite em valores `None` e `NaN`, dependendo do tipo de dado da coluna (valores numéricos serão `NaN`). Nulos podem ser inconvenientes se você não os considerar. Funções de agregação como `SUM`, `MIN`, `MAX`, `COUNT` e `AVG` ignorarão valores nulos, o que podemos aproveitar posteriormente. Ao escrever uma condição `WHERE` em um campo, ela sempre ignorará valores nulos, a menos que você os trate explicitamente. Use os operadores `IS NULL` e `IS NOT NULL` para qualificar/desqualificar valores nulos.

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA
WHERE PHASE_OF_FLIGHT IS NULL
"""

pd.read_sql(sql, conn)

Para esses campos, vamos contar o número de registros e o número de valores não nulos.

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
COUNT(AIRPORT) AS AIRPORT_VALUES, 
COUNT(RUNWAY) AS RUNWAY_VALUES, 
COUNT(PHASE_OF_FLIGHT) AS PHASE_OF_FLIGHT_VALUES, 
COUNT(HEIGHT) AS HEIGHT_VALUES, 
COUNT(SPEED) AS SPEED_VALUES, 
COUNT(DISTANCE) AS DISTANCE_VALUES 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

Se quiséssemos contar o número de valores `NULL` (em vez de valores não `NULL`), poderíamos subtrair a contagem desses campos da contagem de registros ou, mais elegantemente, `SUM` o número de valores verdadeiros usando `IS NULL`. Como `1` será gerado para verdadeiro e `0` para falso, podemos usar isso para somar os valores verdadeiros.

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
SUM(AIRPORT IS NULL) AS AIRPORT_NULLS, 
SUM(RUNWAY IS NULL) AS RUNWAY_NULLS, 
SUM(PHASE_OF_FLIGHT IS NULL) AS PHASE_OF_FLIGHT_NULLS, 
SUM(HEIGHT IS NULL) AS HEIGHT_NULLS, 
SUM(SPEED IS NULL) AS SPEED_NULLS, 
SUM(DISTANCE IS NULL) AS DISTANCE_NULLS 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

Também podemos calcular a porcentagem de nulos em cada um desses campos. Certifique-se apenas de converter a operação para valores de ponto flutuante em vez de inteiros, o que pode ser feito multiplicando a expressão por `1,0`.

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
1.0 * SUM(AIRPORT IS NULL) / COUNT(*) AS AIRPORT_NULL_RATE, 
1.0 * SUM(RUNWAY IS NULL) / COUNT(*)  AS RUNWAY_NULL_RATE, 
1.0 * SUM(PHASE_OF_FLIGHT IS NULL) / COUNT(*) AS PHASE_OF_FLIGHT_NULL_RATE, 
1.0 * SUM(HEIGHT IS NULL) / COUNT(*) AS HEIGHT_NULL_RATE, 
1.0 * SUM(SPEED IS NULL) / COUNT(*) AS SPEED_NULL_RATE, 
1.0 * SUM(DISTANCE IS NULL) / COUNT(*) AS DISTANCE_NULL_RATE 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

Se estivermos interessados ​​em colisões com pássaros que ocorreram abaixo de 500 pés, consideramos os valores nulos e se queremos incluí-los? É por isso que documentar o que `null` significa para um determinado campo é tão importante. Talvez a `HEIGHT` fosse irrelevante ou não pudesse ser medida porque um instrumento estava quebrado. Ou talvez o piloto tenha preenchido o relatório às pressas e não se importado em anotar essas informações. De qualquer forma, precisamos entender por que valores podem estar ausentes e se eles devem ser incluídos em uma determinada análise.

> Um erro comum para iniciantes em SQL é usar `= NULL` em vez de `IS NULL`. Isso não funciona! Sempre use a última opção.

Se quisermos encontrar registros onde `HEIGHT` seja menor que 500 pés, mas quisermos incluir valores `NULL` também, precisamos usar `IS NULL` combinado com `OR` para essa condição.

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE HEIGHT IS NULL OR HEIGHT < 500
"""

pd.read_sql(sql, conn)

Também poderíamos usar uma expressão `CASE` para transformar valores de altura `NULL` em `0`, mas há algo ainda melhor: a função `COALESCE`. Ela pegará um valor possivelmente nulo e o trocará por um valor diferente se for nulo.

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE COALESCE(HEIGHT, 0) < 500
"""

pd.read_sql(sql, conn)

Essa coalescência realiza o mesmo que uma expressão `CASE` convertendo valores nulos.

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE (CASE WHEN HEIGHT IS NULL THEN 0 ELSE HEIGHT END) < 500
"""

pd.read_sql(sql, conn)

Isso fornece todas as ferramentas necessárias para filtrar e manipular valores nulos. Mais tarde, usaremos técnicas para imputar valores ausentes com subconsultas. Não existe uma abordagem única para lidar com valores nulos. Isso sempre dependerá da tarefa e da maneira mais apropriada de lidar com eles.

## Operações com Strings

Grande parte da limpeza de dados envolverá operações com strings. Já vimos como usar curingas com o operador `LIKE`, mas existem muitas funções (e alguns operadores) que visam strings.

| Nome      | Descrição                                                                         |
|-----------|-----------------------------------------------------------------------------------|
| LENGTH    | Conta o número de caracteres em uma string.                                       |
| UPPER     | Converte uma string para letras maiúsculas.                                       |
| LOWER     | Converte uma string para letras minúsculas.                                       |
| SUBSTR    | Extrai uma substring com um comprimento predefinido em uma posição específica.    |
| TRIM      | Remove os caracteres especificados (espaço padrão) do início e do fim da string.  |
| LTRIM     | Remove os caracteres especificados (espaço padrão) do início da string.           |
| RTRIM     | Remove os caracteres especificados (espaço padrão) do final da string.            |
| REPLACE   | Substitui substrings correspondentes em uma string por outra substring.           |
| INSTR     | Retorna a posição da primeira ocorrência da substring, -1 se não for encontrada.  |
| \|\|      | Concatena dois ou mais valores em uma string                                      |
| CONCAT_WS | Concatena várias strings em uma única string com um separador.                    |
| REGEXP    | Determina se uma string corresponde a uma expressão regular.                      | 


Digamos que queremos verificar se todos os valores `AIRPORT_ID` têm quatro caracteres. Podemos usar a função `LENGTH()` para fazer isso e, com certeza, existem alguns que não têm.

In [ ]:
sql = """
SELECT * FROM BIRD_STRIKE_FAA
WHERE LENGTH(AIRPORT_ID) != 4
"""

pd.read_sql(sql, conn)

Aqui está outro exemplo em que substituímos "ARPT" por "AIRPORT" no campo `AIRPORT`.

In [ ]:
sql = """
SELECT 
AIRPORT,
REPLACE(AIRPORT, 'ARPT','AIRPORT') AS AIRPORT_NEW 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

Expressões regulares são definitivamente algo que você vai querer aprender ao limpar dados, e elas são suportadas em SQL, Pandas, Python e muitas outras plataformas. Na verdade, recomendo um [curso Anaconda em Python](https://learning.anaconda.cloud/regular-expressions-in-python). Outras plataformas SQL suportam expressões regulares, mas o SQLite precisa habilitá-las em Python. Podemos fazer isso executando o código abaixo.

In [ ]:
import re

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

conn.create_function("REGEXP", 2, regexp)

Agora, posso procurar as aeronaves Airbus A-320 e A-321 usando uma única expressão regular, como mostrado abaixo.

In [ ]:
sql = """ 
SELECT * FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT REGEXP 'A-32[01]'
"""

pd.read_sql(sql, conn) 

Vamos juntar várias dessas operações com strings para uma tarefa prática. Vamos pegar as colunas `INCIDENT_DATE` e `TIME` e combiná-las em um `DATETIME` adequado. Podemos remover aquele texto padrão "0 days " da coluna `TIME` usando `SUBSTR`. Se o valor de `TIME` estiver ausente, podemos usar `coalesce()` para substituir os valores nulos e transformá-los em "00:00:00". Por fim, concatenamos isso com o `INCIDENT_DATE` e fazemos o cast de tudo como um `DATETIME`.

In [ ]:
sql = """
SELECT OPERATOR, 
AIRCRAFT,
AIRPORT,
DATETIME(INCIDENT_DATE || ' ' || COALESCE(SUBSTR(TIME, 7), '00:00:00')) AS INCIDENT_DATETIME 

FROM BIRD_STRIKE_FAA

ORDER BY INCIDENT_DATE DESC
"""

pd.read_sql(sql, conn)

> **Devo gravar essas alterações de volta na tabela?**
> 
> Uma das vantagens do SQL é que você pode facilmente pegar uma fonte de dados bruta e fazer transformações com consultas SQL. Você pode se perguntar se essas alterações devem ser gravadas de volta na tabela. Eu só faria isso no contexto em que você está realizando o trabalho de extração-transformação-carregamento (ETL) e fornecendo um conjunto de dados limpo para outros. Você pode fazer isso facilmente chamando `CREATE TABLE` com `SELECT` para criar uma nova tabela a partir de uma consulta `SELECT` ou usando `INSERT` com `SELECT` para uma tabela de destino existente. Mas você pode manter todo o seu trabalho de limpeza em consultas executadas conforme necessário, sem precisar armazenar os dados limpos de volta em uma tabela. Considere até mesmo compartilhar as próprias consultas `SELECT`, que podem ser armazenadas em um arquivo de texto, e-mail ou uma visualização.

## Truques com UNION, UNION ALL e CASE

Digamos que eu esteja interessado em comparar o custo de reparos por ano para incidentes abaixo de 1.000 pés e acima de 1.000 pés. A maioria das pessoas usaria `UNION ALL` para anexar essas duas consultas.

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
'BELOW 500' AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
WHERE HEIGHT < 1000 
GROUP BY INCIDENT_YEAR, THRESHOLD

UNION ALL 

SELECT INCIDENT_YEAR, 
'ABOVE 500' AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
WHERE HEIGHT >= 1000 
GROUP BY INCIDENT_YEAR, THRESHOLD
"""

pd.read_sql(sql, conn)

Isso demonstra o `UNION ALL`, que anexa os resultados de ambas as consultas. O `UNION`, que não demonstramos, faria a mesma coisa, mas eliminaria registros duplicados.

No entanto, não sou fã desse caso de uso, por mais comum que seja. As consultas são redundantes e, portanto, precisam executar duas varreduras na tabela, o que é ineficiente. A única diferença entre as consultas é a condição `WHERE`. Se movermos essa condição `WHERE` para uma expressão `CASE`, podemos consolidar em uma única consulta.

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
CASE WHEN HEIGHT < 1000 THEN 'BELOW 1000' ELSE 'ABOVE 1000' END AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR, THRESHOLD
"""

pd.read_sql(sql, conn)

Provavelmente podemos fazer algo ainda melhor aqui. Vamos dividir `TOTAL_REPAIRS` em duas colunas, uma para o limite "ACIMA DE 1000" e outra para o limite "ABAIXO DE 1000". Podemos fazer isso com duas expressões `CASE` dentro das funções `SUM`.

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
SUM(CASE WHEN HEIGHT < 1000 THEN COST_REPAIRS ELSE NULL END) AS BELOW_1000_COST_REPAIRS, 
SUM(CASE WHEN HEIGHT >= 1000 THEN COST_REPAIRS ELSE NULL END) AS ABOVE_1000_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR
"""

pd.read_sql(sql, conn)

Como os valores `NULL` são ignorados por `SUM()` e outras funções de agregação, podemos usá-las para ignorar condicionalmente valores que não queremos contabilizar na soma. Isso efetivamente permite que cada função `SUM()` (ou qualquer função de agregação) tenha diferentes condições `WHERE`.

Como você pode ver, `UNION` e `UNION ALL` nem sempre são uma boa ideia de uso e, muitas vezes, existem maneiras melhores de realizar as tarefas para as quais são comumente usadas, geralmente envolvendo uma expressão `CASE`. Os únicos casos de uso para os quais elas são exclusivamente qualificadas são quando você tem várias consultas extraindo de tabelas diferentes (não da mesma) e transformando-as na mesma saída estrutural a ser anexada.

Use a expressão `CASE` para habilitar padrões poderosos que poucos conhecem!

## Exercício


Encontre o custo total dos reparos por `ANO_INCIDENTE` e `MÊS_INCIDENTE`, mas dividido em dois totais: onde `VELOCIDADE` é menor que 200 e `VELOCIDADE` é maior ou igual a `200`. Complete o código abaixo substituindo os pontos de interrogação por `?`.

In [ ]:
sql = """
SELECT ?, 
?,
? AS BELOW_200_KNOTS_COST_REPAIRS, 
? AS ABOVE_200_KNOTS_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR, INCIDENT_MONTH 
"""

pd.read_sql(sql, conn)

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
INCIDENT_MONTH,
SUM(CASE WHEN SPEED < 200 THEN COST_REPAIRS ELSE NULL END) AS BELOW_200_KNOTS_COST_REPAIRS, 
SUM(CASE WHEN SPEED >= 200 THEN COST_REPAIRS ELSE NULL END) AS ABOVE_200_KNOTS_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY 1, 2
"""

pd.read_sql(sql, conn)